# Bootcamp: Engenheiro(a) de Dados Cloud

### TRABALHO PRÁTICO
### Módulo 1: Fundamentos em Arquitetura de Dados e Soluções em Nuvem


Problema: 

Você é Engenheiro(a) de Dados em uma grande instituição educacional. O gestor de sua área iniciou um novo projeto de inteligência de dados com o objetivo de entender o desempenho de alunos do ensino médio de todo o Brasil no Exame Nacional do Ensino Médio (ENEM). Desse modo, você será responsável por construir um Data Lake com os dados do ENEM 2020, realizar o processamento utilizando ferramental adequado e disponibilizar o dado para consultas dos usuários de negócios e analistas de BI.






 ### 1º FAZER UPLOAD PARA O DATALAKE

1 - Obter as credenciais login e senha para o console da AWS.

In [ ]:
#Imports
import pandas as pd
from pathlib import Path

In [ ]:
#Configuração do caminho das credenciais de acesso.

IAM_user = 'vinicius_perdigao'
path_work = Path('E:\Cursos\Eng_Dados_Cloud')
access_keys_csv = path_work/f'{IAM_user}_accessKeys.csv'


2 -  Configurar o acesso ao AWS CLI.

In [ ]:
#Checar versão do AWS CLI
!aws --version

In [ ]:
# Importar arquivo csv com as credenciais para acessar a AWS.
!aws configure import --csv file://$access_keys_csv

In [ ]:
# Listar os buckets do S3.
!aws s3 ls

3. Fazendo Upload dos arquivos para o Bucket S3.


In [ ]:
print('Fazendo upload de arquivos...')

import boto3

# Criar um cliente para interagir com a AWS S3

s3_client = boto3.client('s3')

# Fazer upload do arquivo para o bucket no s3

s3_client.upload_file("MICRODADOS_ENEM_2020.csv",
                      "datalake-vinicius",
                      "raw-data/MICRODADOS_ENEM_2020.csv")

print('Processo finalizado...')

### Respondendo as perguntas de negócio com querys SQL no SPARK



In [ ]:
# Iniciando uma sessão SPARK
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ENEM_XP").getOrCreate()

In [ ]:
# Ler os dados do enem 2020

enem = (spark.read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .option("delimiter", ";")
    .load("E:\Cursos\Eng_Dados_Cloud\MICRODADOS_ENEM_2020.csv")
    )

In [ ]:
# Print do Schema
enem.printSchema()

In [ ]:
# Separando um DF com as colunas de interesse.
df = (
    enem
    .select("TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT","SG_UF_ESC", "NO_MUNICIPIO_ESC", "NO_MUNICIPIO_PROVA", "Q008", "Q002")

)

In [ ]:
# Printando as 5 primeiras linhas.
df.show(5)

In [ ]:
# Contando o número de participantes no enem 2020
df.count()

In [ ]:
# Criando TempView
df.createOrReplaceTempView("enem")

In [ ]:
# Pergunta 5 - Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?

sqlDF = spark.sql(
                "SELECT round(avg(NU_NOTA_MT),2) as media_matematica \
                FROM enem\
                WHERE TP_SEXO = 'F'")
sqlDF.show()

In [ ]:
# Pergunta - 06 - Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que estudaram numa escola no estado de São Paulo?

sqlDF = spark.sql(
                "SELECT round(avg(NU_NOTA_CH),2) as media_ciencias_humanas \
                FROM enem\
                WHERE TP_SEXO = 'M'\
                AND SG_UF_ESC = 'SP'")
sqlDF.show()

In [ ]:
# Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?

sqlDF = spark.sql(
                "SELECT round(avg(NU_NOTA_CH),2) as media_ciencias_humanas \
                FROM enem\
                WHERE NO_MUNICIPIO_ESC = 'Natal'\
                ")
sqlDF.show()

In [ ]:
# Qual é o município (município da escola) que obteve a maior nota MÉDIA em matemática?

sqlDF = spark.sql(
                "SELECT NO_MUNICIPIO_ESC, avg(NU_NOTA_MT) as medias  \
                FROM enem\
                GROUP BY NO_MUNICIPIO_ESC\
                ORDER BY medias DESC\
                LIMIT 1\
                ")
sqlDF.show()


In [ ]:
# Pergunta 9 - Quantas pessoas que estudaram numa escola em Recife fizeram a prova do ENEM nessa mesma cidade?
sqlDF = spark.sql(
                "SELECT count(*) as quant_pessoas  \
                FROM enem\
                WHERE NO_MUNICIPIO_ESC = 'Recife' \
                AND NO_MUNICIPIO_PROVA = 'Recife'\
                ")
sqlDF.show()


In [ ]:
# PERGUNTA 10 - Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?

sqlDF = spark.sql(
                "SELECT round(avg(NU_NOTA_CH),2) as media_nota  \
                FROM enem\
                WHERE SG_UF_ESC = 'SC' \
                AND Q008 <> 'A'\
                ")
sqlDF.show()




In [ ]:
# Pergunta 11 - Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?

sqlDF = spark.sql(
                "SELECT round(avg(NU_NOTA_MT),2) as media_nota  \
                FROM enem\
                WHERE NO_MUNICIPIO_ESC = 'Belo Horizonte' \
                AND Q002 IN('F','G')\
                AND TP_SEXO = 'F'\
                ")
sqlDF.show()
